In [27]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,GridSearchCV

In [28]:
data = pd.read_csv('diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [29]:
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [30]:
X = data.drop(columns='Outcome')

In [31]:
y = data['Outcome']

In [32]:
## diving the data into training and holdout set
train,val_train,test,val_test = train_test_split(X,y,train_size=0.50,random_state=355)

In [33]:
x_train,x_test,y_train,y_test = train_test_split(train,test,test_size=0.20,random_state=355)

#### we will use knn and svm as our base models

In [34]:
knn = KNeighborsClassifier()
knn.fit(x_train,y_train)

KNeighborsClassifier()

In [35]:
knn.score(x_test,y_test)

0.7402597402597403

In [36]:
svm = SVC()
svm.fit(x_train,y_train)

SVC()

In [37]:
svm.score(x_test,y_test)

0.7402597402597403

#### now we will use the validation set to get the predictions from our base models

In [38]:
predict_val1 = knn.predict(val_train)
predict_val2 = svm.predict(val_train)

In [39]:
## now i am stacking both the prediction data
predict_val_final = np.column_stack((predict_val1,predict_val2))

In [40]:
predict_test1 = knn.predict(x_test)
predict_test2 = svm.predict(x_test)

In [41]:
predict_test_final = np.column_stack((predict_test1,predict_test2))

In [42]:
predict_val_final

array([[0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 1],
       [1, 1],
       [1,

In [43]:
## now i will pass the predict_val_final as the input data along with val_test
rf = RandomForestClassifier()
rf.fit(predict_val_final,val_test)

RandomForestClassifier()

In [44]:
# now we will see the accuracy based on the predict_test and y_test
rf.score(predict_test_final,y_test)

0.7402597402597403

### perform hyperparameter tuning

In [48]:
grid_param = {
    'n_estimators':[90,100,115],
    'criterion':['gini','entropy'],
    'min_samples_leaf':[1,2,3,4,5],
    'min_samples_split':[4,5,6,7,8],
    'max_features':['auto','log2']
}

In [49]:
grid_search = GridSearchCV(estimator=rf,cv=5,n_jobs=-1,param_grid=grid_param,verbose=3)

In [50]:
grid_search.fit(predict_val_final,val_test)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the pas

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the pas

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the pas

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the pas

...

[CV 3/5] END criterion=gini, max_features=auto, min_samples_leaf=1, min_samples_split=6, n_estimators=100;, score=0.805 total time=   0.1s
[CV 1/5] END criterion=gini, max_features=auto, min_samples_leaf=1, min_samples_split=7, n_estimators=90;, score=0.675 total time=   0.1s
[CV 2/5] END criterion=gini, max_features=auto, min_samples_leaf=1, min_samples_split=7, n_estimators=115;, score=0.740 total time=   0.1s
[CV 3/5] END criterion=gini, max_features=auto, min_samples_leaf=1, min_samples_split=7, n_estimators=115;, score=0.805 total time=   0.1s
[CV 5/5] END criterion=gini, max_features=auto, min_samples_leaf=1, min_samples_split=8, n_estimators=115;, score=0.789 total time=   0.1s
[CV 1/5] END criterion=gini, max_features=auto, min_samples_leaf=2, min_samples_split=4, n_estimators=90;, score=0.675 total time=   0.1s
[CV 1/5] END criterion=gini, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=90;, score=0.675 total time=   0.1s
[CV 2/5] END criterion=gini, m

[CV 4/5] END criterion=gini, max_features=auto, min_samples_leaf=2, min_samples_split=8, n_estimators=90;, score=0.701 total time=   0.1s
[CV 5/5] END criterion=gini, max_features=auto, min_samples_leaf=2, min_samples_split=8, n_estimators=90;, score=0.789 total time=   0.1s
[CV 2/5] END criterion=gini, max_features=auto, min_samples_leaf=3, min_samples_split=4, n_estimators=100;, score=0.740 total time=   0.1s
[CV 3/5] END criterion=gini, max_features=auto, min_samples_leaf=3, min_samples_split=4, n_estimators=100;, score=0.805 total time=   0.1s
[CV 4/5] END criterion=gini, max_features=auto, min_samples_leaf=3, min_samples_split=5, n_estimators=90;, score=0.701 total time=   0.1s
[CV 5/5] END criterion=gini, max_features=auto, min_samples_leaf=3, min_samples_split=5, n_estimators=90;, score=0.789 total time=   0.1s
[CV 5/5] END criterion=gini, max_features=auto, min_samples_leaf=3, min_samples_split=6, n_estimators=90;, score=0.789 total time=   0.1s
[CV 1/5] END criterion=gini, max

[CV 3/5] END criterion=gini, max_features=auto, min_samples_leaf=2, min_samples_split=6, n_estimators=100;, score=0.805 total time=   0.1s
[CV 4/5] END criterion=gini, max_features=auto, min_samples_leaf=2, min_samples_split=6, n_estimators=100;, score=0.701 total time=   0.1s
[CV 1/5] END criterion=gini, max_features=auto, min_samples_leaf=2, min_samples_split=7, n_estimators=115;, score=0.675 total time=   0.1s
[CV 2/5] END criterion=gini, max_features=auto, min_samples_leaf=2, min_samples_split=7, n_estimators=115;, score=0.740 total time=   0.1s
[CV 4/5] END criterion=gini, max_features=auto, min_samples_leaf=2, min_samples_split=8, n_estimators=115;, score=0.701 total time=   0.1s
[CV 5/5] END criterion=gini, max_features=auto, min_samples_leaf=2, min_samples_split=8, n_estimators=115;, score=0.789 total time=   0.1s
[CV 1/5] END criterion=gini, max_features=auto, min_samples_leaf=3, min_samples_split=5, n_estimators=100;, score=0.675 total time=   0.1s
[CV 2/5] END criterion=gini

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [4, 5, 6, 7, 8],
                         'n_estimators': [90, 100, 115]},
             verbose=3)

In [51]:
grid_search.best_params_

{'criterion': 'gini',
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 90}

In [52]:
rf_tuned = RandomForestClassifier(criterion='gini',max_features='auto',min_samples_leaf=1,min_samples_split=4,
                                  n_estimators=90)

In [53]:
rf_tuned.fit(predict_val_final,val_test)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_features='auto', min_samples_split=4,
                       n_estimators=90)

In [54]:
rf_tuned.score(predict_test_final,y_test)

0.7402597402597403

[CV 4/5] END criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=8, n_estimators=90;, score=0.701 total time=   0.0s
[CV 5/5] END criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=8, n_estimators=115;, score=0.789 total time=   0.1s
[CV 1/5] END criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=4, n_estimators=90;, score=0.675 total time=   0.1s
[CV 3/5] END criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=90;, score=0.805 total time=   0.1s
[CV 4/5] END criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=90;, score=0.701 total time=   0.1s
[CV 5/5] END criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=115;, score=0.789 total time=   0.1s
[CV 1/5] END criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=6, n_estimators=90;, score=0.675 total time=   0.1s
[CV 1/5] EN

[CV 1/5] END criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=7, n_estimators=115;, score=0.675 total time=   0.1s
[CV 1/5] END criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=8, n_estimators=115;, score=0.675 total time=   0.1s
[CV 2/5] END criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=8, n_estimators=115;, score=0.740 total time=   0.1s
[CV 2/5] END criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=4, n_estimators=115;, score=0.740 total time=   0.1s
[CV 3/5] END criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=4, n_estimators=115;, score=0.805 total time=   0.1s
[CV 3/5] END criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=115;, score=0.805 total time=   0.1s
[CV 4/5] END criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=115;, score=0.701 total time=   0.1s
[CV 3/

[CV 2/5] END criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=4, n_estimators=90;, score=0.740 total time=   0.1s
[CV 5/5] END criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=4, n_estimators=115;, score=0.789 total time=   0.1s
[CV 1/5] END criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=5, n_estimators=90;, score=0.675 total time=   0.1s
[CV 2/5] END criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=6, n_estimators=100;, score=0.740 total time=   0.1s
[CV 3/5] END criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=6, n_estimators=100;, score=0.805 total time=   0.1s
[CV 3/5] END criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=7, n_estimators=100;, score=0.805 total time=   0.1s
[CV 4/5] END criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=7, n_estimators=100;, score=0.701 total time=   0.1s
[CV 2/5]

[CV 1/5] END criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=8, n_estimators=90;, score=0.675 total time=   0.1s
[CV 2/5] END criterion=entropy, max_features=auto, min_samples_leaf=3, min_samples_split=8, n_estimators=90;, score=0.740 total time=   0.1s
[CV 4/5] END criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=4, n_estimators=90;, score=0.701 total time=   0.1s
[CV 5/5] END criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=4, n_estimators=90;, score=0.789 total time=   0.1s
[CV 5/5] END criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=90;, score=0.789 total time=   0.1s
[CV 1/5] END criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=100;, score=0.675 total time=   0.1s
[CV 1/5] END criterion=entropy, max_features=auto, min_samples_leaf=4, min_samples_split=6, n_estimators=100;, score=0.675 total time=   0.1s
[CV 2/5] EN

In [55]:
import pickle

# Save the model to a .sav file
with open('rf_tuned_model_stacking.sav', 'wb') as model_file:
    pickle.dump(rf_tuned, model_file)
